### Configuração Dataframe

In [ ]:
# Instalando as bibliotecas necessárias
!pip install folium
!pip install haversine
!pip install ortools
!pip install geopy

In [2]:
# Importantando as bibliotecas
import pandas as pd
import folium
import folium.plugins as plugins
import plotly.express as px
import random
import requests

from haversine import Unit, haversine_vector
from geopy.distance import geodesic

In [3]:
# Configurando a exibição do Pandas Data
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

In [4]:
# Lendo o Dataframe e visualizando as primeiras linhas
df = pd.read_excel('/content/dados_desafio_rotas.xlsx')

df.head()

,ID,Latitude,Longitude,Tipo_local,Estoque_kg,Estoque_%
0,1,-27.142369,-52.571590,Fábrica,5.000000e+06,100.000000
1,2,-27.168877,-52.572655,Granja,9.137515e+02,5.076397
2,3,-27.105557,-52.541885,Granja,5.419362e+03,30.107565
3,4,-27.088571,-52.522654,Granja,9.743319e+03,54.129550
4,5,-27.091391,-52.473167,Granja,6.943222e+03,38.573455


In [5]:
# Confirmando os tipos das colunas
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53 entries, 0 to 52
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   ID          53 non-null     int64  
 1   Latitude    53 non-null     float64
 2   Longitude   53 non-null     float64
 3   Tipo_local  53 non-null     object 
 4   Estoque_kg  53 non-null     float64
 5   Estoque_%   53 non-null     float64
dtypes: float64(4), int64(1), object(1)
memory usage: 2.6+ KB


### Criando Colunas Necessárias

In [6]:
# Calcular Estoque_capacidade
df['Estoque_capacidade'] = (df['Estoque_kg'] * 100) / df['Estoque_%']

# Calcular Estoque_demanda
df['Estoque_demanda'] = df['Estoque_capacidade'] - df['Estoque_kg']

# Identificando as unidades com o estoque mais baixo
df.sort_values(by='Estoque_demanda', ascending=True).head(11)

,ID,Latitude,Longitude,Tipo_local,Estoque_kg,Estoque_%,Estoque_capacidade,Estoque_demanda
0,1,-27.142369,-52.571590,Fábrica,5.000000e+06,100.000000,5000000.0,0.000000
34,35,-27.173656,-52.475296,Granja,1.782429e+04,99.023819,18000.0,175.712523
27,28,-27.173384,-52.540278,Granja,1.778243e+04,98.791293,18000.0,217.567180
30,31,-27.155354,-52.485992,Granja,1.753277e+04,97.404280,18000.0,467.229532
42,43,-27.155032,-52.429955,Granja,1.714951e+04,95.275060,18000.0,850.489240
28,29,-27.099042,-52.477391,Granja,1.657987e+04,92.110384,18000.0,1420.130965
39,40,-27.178827,-52.458685,Granja,1.648845e+04,91.602493,18000.0,1511.551324
35,36,-27.113452,-52.558373,Granja,1.488227e+04,82.679303,18000.0,3117.725538
6,7,-27.181639,-52.466873,Granja,1.381475e+04,76.748597,18000.0,4185.252505
19,20,-27.108596,-52.435166,Granja,1.380293e+04,76.682933,18000.0,4197.072130


In [7]:
# Calcular a matriz de distâncias
def calculate_distance(row):
    fabrica = (df.loc[0, 'Latitude'], df.loc[0, 'Longitude'])
    granja = (row['Latitude'], row['Longitude'])
    return geodesic(fabrica, granja).kilometers

df['Dist_Fabrica'] = df.apply(calculate_distance, axis=1)

df.head()

,ID,Latitude,Longitude,Tipo_local,Estoque_kg,Estoque_%,Estoque_capacidade,Estoque_demanda,Dist_Fabrica
0,1,-27.142369,-52.571590,Fábrica,5.000000e+06,100.000000,5000000.0,0.000000,0.000000
1,2,-27.168877,-52.572655,Granja,9.137515e+02,5.076397,18000.0,17086.248485,2.939040
2,3,-27.105557,-52.541885,Granja,5.419362e+03,30.107565,18000.0,12580.638315,5.031126
3,4,-27.088571,-52.522654,Granja,9.743319e+03,54.129550,18000.0,8256.681086,7.686272
4,5,-27.091391,-52.473167,Granja,6.943222e+03,38.573455,18000.0,11056.778165,11.275727


### Visualizando Estatística Básica

In [8]:
# Visualizando as unidades com o estoque baixo
fig = px.bar(df, y='Estoque_%', x='ID', text='Estoque_kg', title='Nível dos Estoques das Unidades (%)')
fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig.update_layout(uniformtext_minsize=8, uniformtext_mode='hide')
fig.show()

In [9]:
# Estatística descritiva básica sobre as distâncias
df['Dist_Fabrica'].describe()

count    53.000000
mean      9.098350
std       3.859532
min       0.000000
25%       6.954456
50%       9.384121
75%      11.897218
max      15.309226
Name: Dist_Fabrica, dtype: float64

In [10]:
# Identificando as Granjas mais próximas à Fábrica
df.sort_values(by='Dist_Fabrica', ascending=True)

,ID,Latitude,Longitude,Tipo_local,Estoque_kg,Estoque_%,Estoque_capacidade,Estoque_demanda,Dist_Fabrica
0,1,-27.142369,-52.571590,Fábrica,5.000000e+06,100.000000,5000000.0,0.000000,0.000000
50,51,-27.131966,-52.552221,Granja,1.216591e+04,67.588379,18000.0,5834.091772,2.239554
25,26,-27.163537,-52.571551,Granja,4.878496e+03,27.102757,18000.0,13121.503767,2.345495
1,2,-27.168877,-52.572655,Granja,9.137515e+02,5.076397,18000.0,17086.248485,2.939040
13,14,-27.145678,-52.542008,Granja,4.773079e+03,26.517104,18000.0,13226.921354,2.955151
35,36,-27.113452,-52.558373,Granja,1.488227e+04,82.679303,18000.0,3117.725538,3.461762
26,27,-27.172752,-52.553907,Granja,1.244174e+04,69.120770,18000.0,5558.261323,3.795410
24,25,-27.118236,-52.535388,Granja,1.118615e+04,62.145273,18000.0,6813.850939,4.475726
27,28,-27.173384,-52.540278,Granja,1.778243e+04,98.791293,18000.0,217.567180,4.630484
2,3,-27.105557,-52.541885,Granja,5.419362e+03,30.107565,18000.0,12580.638315,5.031126


In [11]:
import folium.plugins as plugins
# Visualizando a espacialização das unidades
mapa = folium.Map(location = [-27.176622253334653, -52.47969115683378],
                  zoom_start=12,
                  width="%95",
                  height="%95")

url = "https://servicodados.ibge.gov.br/api/v3/malhas/estados/SC?formato=application/vnd.geo+json"
geo_json_data = requests.get(url).json()

style =  {'fillColor': 'blue', #cor de preenchimento
              'color': 'red',#cor da linha de contorno
             'weight': 0.0, #espessura da linha
               }

folium.GeoJson(geo_json_data, style_function=lambda x:style).add_to(mapa)

for index, row in df.iterrows():
  folium.Marker([row['Latitude'], row['Longitude']],
                      radius=row['Estoque_%'],
                      color="black",
                      weight=row['Estoque_%']/100,
                      fill_opacity=0.6,
                      fill_color="green",
                      fill=False,  # gets overridden by fill_color
                      opacity=1,
                      popup=row['ID']/75,
                      tooltip=f"Unidade {row['ID']} está com {row['Estoque_%']} de estoque.",
                    ).add_to(mapa)

minimap = folium.plugins.MiniMap()
mapa.add_child(minimap)

mapa

### Construindo Matriz Distância x Estoque

In [12]:
# Criando índice de eficiência para rankear as prioridades de entrega de forma mais eficiente
df['Indice_eficiencia'] = df['Estoque_demanda'] / df['Dist_Fabrica']
df.sort_values(by='Indice_eficiencia', ascending=False)

,ID,Latitude,Longitude,Tipo_local,Estoque_kg,Estoque_%,Estoque_capacidade,Estoque_demanda,Dist_Fabrica,Indice_eficiencia
1,2,-27.168877,-52.572655,Granja,9.137515e+02,5.076397,18000.0,17086.248485,2.939040,5813.548067
25,26,-27.163537,-52.571551,Granja,4.878496e+03,27.102757,18000.0,13121.503767,2.345495,5594.344040
13,14,-27.145678,-52.542008,Granja,4.773079e+03,26.517104,18000.0,13226.921354,2.955151,4475.887035
31,32,-27.189097,-52.545921,Granja,2.914516e+03,16.191756,18000.0,15085.483830,5.768918,2614.958852
50,51,-27.131966,-52.552221,Granja,1.216591e+04,67.588379,18000.0,5834.091772,2.239554,2605.024408
2,3,-27.105557,-52.541885,Granja,5.419362e+03,30.107565,18000.0,12580.638315,5.031126,2500.561114
29,30,-27.165091,-52.504577,Granja,1.972261e+03,10.957004,18000.0,16027.739321,7.103415,2256.342666
33,34,-27.202284,-52.528153,Granja,1.297671e+03,7.209286,18000.0,16702.328595,7.912318,2110.927282
51,52,-27.171884,-52.505242,Granja,3.034224e+03,16.856800,18000.0,14965.776038,7.344452,2037.698061
7,8,-27.186685,-52.513516,Granja,2.991424e+03,16.619023,18000.0,15008.575921,7.565721,1983.760177


### Construindo grupos de entrega para cada rota

In [13]:
# Ordenar os locais por índice de eficiência em ordem decrescente
df = df.sort_values(by='Indice_eficiencia', ascending=False)

# Definir capacidade de entrega de um caminhão para cada grupo (em kg)
limite_racao_grupo = 10000

# Criar grupos de locais de entrega respeitando o limite de ração e garantindo que as granjas sejam completamente abastecidas
grupos = []
grupo_atual = []
racao_total_grupo = 0
capacidade_caminhao = limite_racao_grupo

for index, row in df.iterrows():
    # Verificar se a granja cabe no grupo e se a capacidade do caminhão não foi excedida
    if row['Estoque_kg'] <= capacidade_caminhao:
        grupo_atual.append(index)
        racao_total_grupo += row['Estoque_kg']
        capacidade_caminhao -= row['Estoque_kg']
    else:
        # Adicionar grupo atual à lista de grupos
        grupos.append(grupo_atual.copy())  # Usar .copy() para evitar alterações inadvertidas

        # Iniciar novo grupo com a granja atual
        grupo_atual = [index]
        racao_total_grupo = row['Estoque_kg']
        capacidade_caminhao = limite_racao_grupo - row['Estoque_kg']

# Adicionar último grupo à lista de grupos
grupos.append(grupo_atual.copy())  # Usar .copy() para evitar alterações inadvertidas

# Simular a entrega de cada grupo
for i, grupo in enumerate(grupos, start=1):
    capacidade_caminhao = limite_racao_grupo
    for granja in grupo:
        estoque_granja = df.loc[granja, 'Estoque_kg']  # Acessa o valor de estoque da granja
        if estoque_granja <= capacidade_caminhao:  # Comparação direta com a capacidade do caminhão
            capacidade_caminhao -= estoque_granja
            print(f'Granja {granja} abastecida')
        else:
            print(f'Caminhão cheio, retornando à fábrica')
            break
    print(f'Fim do Grupo {i}')


Granja 1 abastecida
Granja 25 abastecida
Fim do Grupo 1
Granja 13 abastecida
Granja 31 abastecida
Fim do Grupo 2
Caminhão cheio, retornando à fábrica
Fim do Grupo 3
Granja 2 abastecida
Granja 29 abastecida
Granja 33 abastecida
Fim do Grupo 4
Granja 51 abastecida
Granja 7 abastecida
Granja 22 abastecida
Fim do Grupo 5
Granja 12 abastecida
Fim do Grupo 6
Granja 32 abastecida
Fim do Grupo 7
Granja 14 abastecida
Granja 20 abastecida
Fim do Grupo 8
Caminhão cheio, retornando à fábrica
Fim do Grupo 9
Caminhão cheio, retornando à fábrica
Fim do Grupo 10
Caminhão cheio, retornando à fábrica
Fim do Grupo 11
Granja 52 abastecida
Fim do Grupo 12
Granja 3 abastecida
Fim do Grupo 13
Granja 38 abastecida
Fim do Grupo 14
Granja 4 abastecida
Fim do Grupo 15
Granja 23 abastecida
Fim do Grupo 16
Granja 18 abastecida
Fim do Grupo 17
Granja 17 abastecida
Fim do Grupo 18
Granja 16 abastecida
Fim do Grupo 19
Granja 37 abastecida
Fim do Grupo 20
Granja 47 abastecida
Fim do Grupo 21
Caminhão cheio, retornando

### Mapa das rotas de entrega

In [14]:
# Criando o mapa das rotas das entregas à partir da fábrica
mapa = folium.Map(location=[-27.176622253334653, -52.47969115683378],
                  zoom_start=13,
                  width="%95",
                  height="%95")

# Adicionar marcador para a fábrica
folium.Marker(
    location=[-27.142369, -52.571590],
    popup="Fábrica",
    icon=folium.Icon(color='green', icon='industry', prefix='fa')
).add_to(mapa)

# Adicionar marcadores para cada granja
for index, row in df.iterrows():
    if row['Tipo_local'] == 'Granja':
        folium.Marker(
            location=[row['Latitude'], row['Longitude']],
            popup=f"Granja {index}",
            icon=folium.Icon(color='blue', icon='cow', prefix='fa')
        ).add_to(mapa)

# Adicionar polylines para cada grupo de entrega no mapa com cores aleatórias
for i, grupo in enumerate(grupos, start=1):
    grupo_coords = [[-27.142369, -52.571590]]  # Coordenadas iniciais na fábrica
    grupo_coords.extend([[df.loc[granja, 'Latitude'], df.loc[granja, 'Longitude']] for granja in grupo])
    grupo_coords.append([-27.142369, -52.571590])  # Coordenadas finais na fábrica
    cor = "#{:06x}".format(random.randint(0, 0xFFFFFF))  # Gerar cor aleatória
    folium.PolyLine(
        locations=grupo_coords,
        color=cor,
        radius=10,
        weight=4,
        opacity=0.7,
        tooltip=f'Grupo {i}'
    ).add_to(mapa)

minimap = plugins.MiniMap()
mapa.add_child(minimap)

# Exibir o mapa
mapa

### Listagem dod grupos de entrega por rota

In [15]:
x = 1
for i in grupos:
  print(f'Rota {x} formada pelas Granjas: {i}')
  x += 1


Rota 1 formada pelas Granjas: [1, 25]
Rota 2 formada pelas Granjas: [13, 31]
Rota 3 formada pelas Granjas: [50]
Rota 4 formada pelas Granjas: [2, 29, 33]
Rota 5 formada pelas Granjas: [51, 7, 22]
Rota 6 formada pelas Granjas: [12]
Rota 7 formada pelas Granjas: [32]
Rota 8 formada pelas Granjas: [14, 20]
Rota 9 formada pelas Granjas: [24]
Rota 10 formada pelas Granjas: [26]
Rota 11 formada pelas Granjas: [11]
Rota 12 formada pelas Granjas: [52]
Rota 13 formada pelas Granjas: [3]
Rota 14 formada pelas Granjas: [38]
Rota 15 formada pelas Granjas: [4]
Rota 16 formada pelas Granjas: [23]
Rota 17 formada pelas Granjas: [18]
Rota 18 formada pelas Granjas: [17]
Rota 19 formada pelas Granjas: [16]
Rota 20 formada pelas Granjas: [37]
Rota 21 formada pelas Granjas: [47]
Rota 22 formada pelas Granjas: [35]
Rota 23 formada pelas Granjas: [45]
Rota 24 formada pelas Granjas: [8]
Rota 25 formada pelas Granjas: [9]
Rota 26 formada pelas Granjas: [43]
Rota 27 formada pelas Granjas: [48]
Rota 28 formada 